In [1]:
import json,requests,dateutil
from datetime import datetime,timedelta
import pandas as pd
from tqdm import tqdm
from requests.auth import HTTPBasicAuth
configFile='../Data/CONFIG.json'
inputFile='../Data/input_data.csv'
getConfig=json.loads(open(configFile,'r').read())
getConfig

{'c_url': 'https://mlw.basic.stage.c8y.io/',
 'c_user': 'vran',
 'c_pass': 'Testing@1234',
 'c_device_source': '288576'}

In [2]:
data=pd.read_csv(inputFile)
data.columns=['timeStamp','c8y_Flow']
print (data.shape)
data.head()

(97, 2)


timeStamp    c8y_Flow
0  2019-12-01 00:00:00   27.666667
1  2019-12-01 02:00:00   27.250000
2  2019-12-01 04:00:00   52.500000
3  2019-12-01 06:00:00   99.750000
4  2019-12-01 08:00:00  151.750000

In [3]:
data['timeStamp']=data['timeStamp'].apply(lambda x: dateutil.parser.parse(x))
cDate=datetime.now()
data['dates']=data['timeStamp'].apply(lambda x: x.date())
data['times']=data['timeStamp'].apply(lambda x: x.time())

In [6]:
newDf=pd.DataFrame(pd.unique(data['dates']),columns=['dates'])
newDf['index']=list(range(1,newDf.shape[0]+1))[::-1]
newDf['newDate']=newDf['index'].apply(lambda x: cDate.date()-timedelta(days=x))
data=pd.merge(data,newDf[['dates','newDate']],on=['dates'],how='left')
# data['datetimeToUse']=data[['newDate','times']].apply(lambda x: datetime.combine(*x),axis=1)

In [7]:
import requests
import json
auth = HTTPBasicAuth(getConfig['c_user'], getConfig['c_pass'])
url = getConfig['c_url']+"measurement/measurements"
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/vnd.com.nsn.cumulocity.measurement+json'
}


for i in tqdm(range(data.shape[0])):
    values=data.iloc[i]
    payload = json.dumps({
                          "c8y_Flow": {"F": {"unit": "psi","value": values['c8y_Flow']}},
                          "time": str(values['newDate'])+'T'+str(values['times'])+'.000+05:30',
                          "source": {
                            "id": getConfig['c_device_source']
                          },
                          "type": "c8y_Flow"
                        })
    response = requests.post( url,auth=auth, headers=headers, data=payload)


  0%|          | 0/97 [00:00<?, ?it/s]

  2%|▏         | 2/97 [00:00<00:05, 18.12it/s]

  5%|▌         | 5/97 [00:00<00:04, 19.44it/s]

  8%|▊         | 8/97 [00:00<00:04, 21.29it/s]

 11%|█▏        | 11/97 [00:00<00:03, 23.20it/s]

...

 89%|████████▊ | 86/97 [00:03<00:00, 28.86it/s]

 92%|█████████▏| 89/97 [00:03<00:00, 28.99it/s]

 95%|█████████▍| 92/97 [00:03<00:00, 28.47it/s]

 98%|█████████▊| 95/97 [00:03<00:00, 27.53it/s]

100%|██████████| 97/97 [00:03<00:00, 27.49it/s]